# Financial Q&A Systems - RAG System Implementation

This notebook demonstrates the implementation of the Retrieval-Augmented Generation (RAG) system for financial Q&A.


## Setup and Imports


In [ ]:
import os
import sys
import json
import time
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add the project root to the path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import project modules with enhanced error handling
print("🔄 Loading RAG system modules...")
imports_successful = True

try:
    from src.rag_system.document_chunker import DocumentChunker
    print("✅ DocumentChunker imported")
except ImportError as e:
    print(f"❌ DocumentChunker import failed: {e}")
    imports_successful = False

try:
    from src.rag_system.embedding_manager import EmbeddingManager
    print("✅ EmbeddingManager imported")
except ImportError as e:
    print(f"❌ EmbeddingManager import failed: {e}")
    print("⚠️ This may be due to sentence_transformers/transformers version conflict")
    print("💡 Try: pip install --upgrade sentence_transformers transformers")
    imports_successful = False

try:
    from src.rag_system.answer_generator import AnswerGenerator
    print("✅ AnswerGenerator imported")
except ImportError as e:
    print(f"❌ AnswerGenerator import failed: {e}")
    imports_successful = False

try:
    from src.rag_system.integrated_rag import IntegratedRAG as RAG
    print("✅ IntegratedRAG imported")
except ImportError as e:
    print(f"❌ IntegratedRAG import failed: {e}")
    imports_successful = False

if imports_successful:
    print("🎉 All RAG imports successful!")
else:
    print("❌ Some imports failed. The notebook may not function correctly.")
    print("📝 Check your environment and package versions.")


/opt/anaconda3/envs/CAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'GGUF_CONFIG_MAPPING' from 'transformers.integrations' (/opt/anaconda3/envs/CAI/lib/python3.10/site-packages/transformers/integrations/__init__.py)

## Define Paths


In [ ]:
# Define paths
DATA_DIR = project_root / "data"
PROCESSED_DIR = DATA_DIR / "processed"
QA_PAIRS_DIR = DATA_DIR / "qa_pairs"
CHUNKS_DIR = DATA_DIR / "chunks"
INDEXES_DIR = DATA_DIR / "indexes"
RAG_MODEL_DIR = project_root / "models" / "rag"

# Create directories if they don't exist
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
QA_PAIRS_DIR.mkdir(parents=True, exist_ok=True)
CHUNKS_DIR.mkdir(parents=True, exist_ok=True)
INDEXES_DIR.mkdir(parents=True, exist_ok=True)
RAG_MODEL_DIR.mkdir(parents=True, exist_ok=True)


## Step 1: Document Chunking

In this step, we'll chunk the processed documents into smaller segments for retrieval.


In [ ]:
# Initialize the document chunker (only if imports were successful)
if imports_successful and 'DocumentChunker' in globals():
    try:
        chunker = DocumentChunker(chunk_sizes=[100, 400], chunk_overlap=50)
        print("✅ Document chunker initialized")
        
        # List processed files
        processed_files = list(PROCESSED_DIR.glob("*.txt"))
        print(f"Found {len(processed_files)} processed files:")
        for file in processed_files:
            print(f"  - {file.name}")
    except Exception as e:
        print(f"❌ Error initializing chunker: {e}")
        chunker = None
        processed_files = []
else:
    print("❌ Cannot initialize chunker - imports failed")
    chunker = None
    processed_files = []


In [ ]:
# Chunk each processed file
all_chunks = {}

if processed_files:
    for file_path in processed_files:
        try:
            print(f"Chunking {file_path.name}...")
            chunks_by_size = chunker.chunk_file(file_path, output_dir=CHUNKS_DIR)
            all_chunks[file_path.stem] = chunks_by_size
            
            # Print statistics for each chunk size
            for size, chunks in chunks_by_size.items():
                print(f"  - Size {size}: {len(chunks)} chunks")
        except Exception as e:
            print(f"  - Error chunking {file_path.name}: {e}")
            import traceback
            traceback.print_exc()
else:
    print("❌ No processed files found. Please run the data preprocessing notebook first.")
    print("Creating sample chunks for demonstration...")
    
    # Create sample chunks if no files are available
    sample_chunks = [
        {
            "text": "ACME Corporation reported revenue of $1,250 million for fiscal year 2023, representing a 12% increase from the previous year.",
            "chunk_id": "sample_1_chunk_1",
            "source": "sample_document",
            "chunk_size": 100
        },
        {
            "text": "The company's net profit margin improved to 15% in 2023, up from 14.4% in 2022, driven by operational efficiency improvements.",
            "chunk_id": "sample_1_chunk_2", 
            "source": "sample_document",
            "chunk_size": 100
        },
        {
            "text": "Total assets increased to $3,500 million, with current assets of $1,800 million and non-current assets of $1,700 million.",
            "chunk_id": "sample_1_chunk_3",
            "source": "sample_document", 
            "chunk_size": 100
        }
    ]
    
    all_chunks = {"sample_document": {400: sample_chunks}}
    print(f"Created {len(sample_chunks)} sample chunks")


In [ ]:
# Analyze chunk length distribution
chunk_lengths = []
for file_name, chunks_by_size in all_chunks.items():
    # Use the larger chunk size (400 tokens)
    if 400 in chunks_by_size:
        for chunk in chunks_by_size[400]:
            chunk_lengths.append(len(chunk["text"]))

plt.figure(figsize=(10, 6))
sns.histplot(chunk_lengths, kde=True)
plt.title("Chunk Length Distribution (400 token chunks)")
plt.xlabel("Length (characters)")
plt.ylabel("Count")
plt.show()

print(f"Total chunks: {len(chunk_lengths)}")
print(f"Average chunk length: {sum(chunk_lengths) / len(chunk_lengths):.1f} characters")
print(f"Min chunk length: {min(chunk_lengths)} characters")
print(f"Max chunk length: {max(chunk_lengths)} characters")


## Step 2: Create Embeddings and Indexes

Now, we'll create embeddings and indexes for efficient retrieval.


In [ ]:
# Initialize the embedding manager
try:
    embedding_manager = EmbeddingManager(model_name="all-MiniLM-L6-v2")
    print("✅ Embedding manager initialized")
except Exception as e:
    print(f"❌ Error initializing embedding manager: {e}")
    embedding_manager = None

if embedding_manager:
    # Load chunks (using the larger chunk size for better context)
    chunk_size = 400
    all_file_chunks = []
    
    # Combine chunks from all files
    for file_name, chunks_by_size in all_chunks.items():
        if chunk_size in chunks_by_size:
            # Add file name to each chunk for tracking
            for chunk in chunks_by_size[chunk_size]:
                chunk["file"] = file_name
            
            all_file_chunks.extend(chunks_by_size[chunk_size])
    
    print(f"Total chunks to embed: {len(all_file_chunks)}")
    
    # Build indexes
    if all_file_chunks:
        try:
            print("Building indexes...")
            start_time = time.time()
            embedding_manager.build_indexes(all_file_chunks)
            end_time = time.time()
            print(f"✅ Indexes built in {end_time - start_time:.2f} seconds")
        except Exception as e:
            print(f"❌ Error building indexes: {e}")
            print("This is expected if LangChain components have compatibility issues.")
            print("The system will fall back to simpler retrieval methods.")
    else:
        print("❌ No chunks available for indexing")
else:
    print("❌ Cannot proceed without embedding manager")

In [ ]:
# Save indexes for future use
print("Saving indexes...")
embedding_manager.save_indexes(INDEXES_DIR)
print(f"Indexes saved to {INDEXES_DIR}")


## Step 3: Test Retrieval Methods

Let's test different retrieval methods (dense, sparse, hybrid) to see which performs best.


In [ ]:
# Test queries
test_queries = [
    "What was the revenue in the last fiscal year?",
    "How much profit did the company make?",
    "What are the total assets?",
    "What is the debt to equity ratio?",
    "How did sales compare to the previous year?"
]

# Compare retrieval methods
retrieval_methods = ["dense", "sparse", "hybrid"]
top_k = 3

# Function to display retrieval results
def display_retrieval_results(query, results):
    print(f"Query: {query}")
    print("-" * 80)
    
    for i, result in enumerate(results):
        try:
            print(f"Result {i+1} (Score: {result['score']:.4f}, Method: {result['method']}):")
            print(f"File: {result['chunk'].get('file', 'unknown')}")
            print(f"Text: {result['chunk']['text'][:200]}...")
        except Exception as e:
            print(f"Error displaying result {i+1}: {e}")
        print()
    
    print("=" * 80)

# Test each query with each retrieval method
for query in test_queries:
    print(f"\nTesting query: {query}")

## Step 4: Set Up Answer Generation

Now, let's set up the answer generation component using a small language model.


In [ ]:
# Initialize the answer generator
print("Initializing answer generator...")
answer_generator = AnswerGenerator(model_name="distilgpt2")
print("Answer generator initialized")


In [ ]:
# Test answer generation
test_query = "What was the revenue in the last fiscal year?"

# Get retrieved chunks using hybrid retrieval
retrieved_chunks = embedding_manager.hybrid_search(test_query, top_k=3)

# Generate answer
print(f"Generating answer for: {test_query}")
try:
    answer, confidence, response_time = answer_generator.generate_answer(test_query, retrieved_chunks)

    print(f"\nAnswer: {answer}")
    print(f"Confidence: {confidence:.2f}")
    print(f"Response time: {response_time:.3f}s")

    # Apply guardrails
    modified_answer, is_hallucination = answer_generator.apply_guardrails(test_query, answer, retrieved_chunks)

    if is_hallucination:
        print("\nHallucination detected! Modified answer:")
        print(modified_answer)
    else:
        print("\nNo hallucination detected.")
except Exception as e:
    print(f"Error generating answer: {e}")

## Step 5: Integrate the RAG System

Finally, let's integrate all components into the complete RAG system.


In [ ]:
# Initialize the RAG system
print("Initializing RAG system...")
rag_system = RAG(
    embedding_model="all-MiniLM-L6-v2",
    llm_model="distilgpt2",
    chunk_sizes=[100, 400],
    chunk_overlap=50,
    retrieval_method="hybrid",
    top_k=3
)

# We'll use the embedding_manager and answer_generator we've already set up
rag_system.embedding_manager = embedding_manager
rag_system.answer_generator = answer_generator
rag_system.is_initialized = True

print("RAG system initialized")


In [ ]:
# Test the RAG system with sample queries
test_queries = [
    "What was the revenue in the last fiscal year?",
    "How much profit did the company make?",
    "What are the total assets?",
    "What is your favorite color?"  # Irrelevant query to test guardrails
]

for query in test_queries:
    print(f"\nProcessing query: {query}")
    print("-" * 80)
    
    try:
        result = rag_system.process_query(query)
        
        print(f"Answer: {result['answer']}")
        print(f"Confidence: {result['confidence']:.2f}")
        print(f"Response time: {result['response_time']:.3f}s")
        
        if result['is_filtered']:
            print("Query was filtered by input-side guardrails")
        elif result.get('is_hallucination', False):
            print("Hallucination detected by output-side guardrails")
    except Exception as e:
        print(f"Error processing query: {e}")
    
    print("=" * 80)

## Step 6: Save the RAG System

Let's save the complete RAG system for later use.


In [ ]:
# Save the RAG system
print("Saving RAG system...")
rag_system.save(RAG_MODEL_DIR)
print(f"RAG system saved to {RAG_MODEL_DIR}")


## Summary

In this notebook, we've implemented the complete RAG system for financial Q&A:

1. Chunked the processed documents into smaller segments
2. Created embeddings and indexes for efficient retrieval
3. Tested different retrieval methods (dense, sparse, hybrid)
4. Set up answer generation with a small language model
5. Integrated all components into a complete RAG system
6. Saved the system for later use

The RAG system is now ready for evaluation and comparison with the Fine-Tuned model.
